In [1]:
import pandas as pd
message = pd.read_csv("SPAM text message 20170820 - Data.csv")

In [2]:
message

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [4]:
import re
corpus = []
for i in range(0, len(message)):
    review = re.sub(r'[^a-zA-Z]',' ', message['Message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus.append(review)

In [5]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [6]:
words = []
for sent in corpus:
    sent_token = sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [7]:
#Train word2vec 

import gensim
model = gensim.models.Word2Vec(words, vector_size = 100)

In [8]:
#checking the words and explore
model.wv.similar_by_word('good')

[('day', 0.9990753531455994),
 ('well', 0.9989384412765503),
 ('dear', 0.9988654255867004),
 ('oh', 0.9988046288490295),
 ('hope', 0.9987967014312744),
 ('all', 0.9986788630485535),
 ('wish', 0.9986740946769714),
 ('and', 0.9986687302589417),
 ('happy', 0.9986634254455566),
 ('amp', 0.9986497163772583)]

In [9]:
model.wv['good'].shape

(100,)

In [10]:
import numpy as np

def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis = 0)

In [11]:
from tqdm import tqdm
X = []
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  0%|                                                                                         | 0/5569 [00:00<?, ?it/s]C:\Users\Sourav_Moutushi\anaconda3\envs\pandas_playground\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Sourav_Moutushi\anaconda3\envs\pandas_playground\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|████████████████████████████████████████████████████████████████████████████| 5569/5569 [00:00<00:00, 7223.10it/s]


In [12]:
words[0]

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

In [13]:
X[0]

array([-0.1504512 ,  0.21225196,  0.08721492,  0.08323719,  0.05605296,
       -0.53062177,  0.17376588,  0.5700318 , -0.27762446, -0.14372571,
       -0.13229409, -0.3601748 , -0.04908351,  0.1195132 ,  0.23970045,
       -0.21513717,  0.0792217 , -0.29942307, -0.07377852, -0.5409934 ,
        0.18255523,  0.15536581,  0.1150948 , -0.22484252, -0.05238384,
        0.05572136, -0.28593886, -0.25604102, -0.2163488 ,  0.01817123,
        0.32094175, -0.01505701,  0.07349954, -0.11258936, -0.10216948,
        0.38079682, -0.0743295 , -0.3004416 , -0.2304465 , -0.3755305 ,
        0.12768534, -0.1784762 , -0.15762848, -0.08247838,  0.23713924,
       -0.02027053, -0.15944776, -0.17816561,  0.14205144,  0.12468175,
        0.19094072, -0.17908007, -0.06219193,  0.13157524, -0.16690117,
        0.18485203,  0.20791772,  0.09409044, -0.24473287,  0.15729338,
        0.05128595,  0.08927771,  0.0159827 , -0.12092744, -0.2665652 ,
        0.26420575,  0.10112526,  0.26918104, -0.40409318,  0.29

In [14]:
type(X)

list

In [15]:
X_new = np.array(X, dtype=object)
X_new.shape

(5569,)

In [16]:
message.shape

(5572, 2)

In [17]:
y = message[list(map(lambda x: len(x)>0, corpus))]
y = pd.get_dummies(y['Category'])
print(y)

      ham  spam
0       1     0
1       1     0
2       0     1
3       1     0
4       1     0
...   ...   ...
5567    0     1
5568    1     0
5569    1     0
5570    1     0
5571    1     0

[5569 rows x 2 columns]


In [18]:
y = y.iloc[:,0].values
print(y)

[1 1 0 ... 1 1 1]


In [19]:
X[0].shape

(100,)

In [20]:
X[0].reshape(1,-1).shape

(1, 100)

In [21]:
df = pd.DataFrame()
for i in range(0, len(X)):
    df = pd.concat([df.astype('float64'), pd.DataFrame(X[i].reshape(1,-1))], ignore_index = True)

In [22]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.150451,0.212252,0.087215,0.083237,0.056053,-0.530622,0.173766,0.570032,-0.277624,-0.143726,...,0.357765,0.139875,0.031628,0.039413,0.417188,0.283252,0.089710,-0.221075,0.135671,-0.007960
1,-0.137281,0.188564,0.063627,0.075256,0.048973,-0.457554,0.146796,0.483996,-0.232966,-0.117185,...,0.306303,0.116510,0.023187,0.029506,0.350185,0.244428,0.077006,-0.202106,0.124122,-0.000106
2,-0.127038,0.219591,0.102723,0.108793,0.063124,-0.603604,0.169922,0.600346,-0.295312,-0.132415,...,0.378375,0.141442,0.019797,0.035582,0.428247,0.275481,0.061758,-0.227003,0.134733,-0.010629
3,-0.217171,0.294801,0.110569,0.117163,0.068825,-0.713371,0.242240,0.772965,-0.378703,-0.192415,...,0.484549,0.184006,0.037733,0.051443,0.561150,0.390037,0.130860,-0.309862,0.185871,0.001926
4,-0.191087,0.236407,0.105237,0.098151,0.071505,-0.610927,0.200266,0.658831,-0.320140,-0.168052,...,0.420381,0.149794,0.040161,0.044856,0.478728,0.337393,0.101422,-0.271695,0.158725,-0.006219


In [23]:
df['output'] = y

In [24]:
y

array([1, 1, 0, ..., 1, 1, 1], dtype=uint8)

In [25]:
print(df.isnull().sum())
df.dropna(inplace=True)

0         12
1         12
2         12
3         12
4         12
          ..
96        12
97        12
98        12
99        12
output     0
Length: 101, dtype: int64


In [26]:
X = df.drop("output", axis=1)
y = df['output']

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.02)

In [28]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

In [29]:
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [30]:
y_pred = classifier.predict(X_test)

In [31]:
y_pred

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=uint8)

In [32]:
y_test

1184    1
775     1
496     1
1655    1
4993    1
       ..
996     1
578     1
2150    1
5181    1
4402    1
Name: output, Length: 112, dtype: uint8

In [33]:
from sklearn.metrics import accuracy_score, classification_report
acc = accuracy_score(y_test,y_pred)
print(acc)

0.9553571428571429


In [34]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.87      0.84        15
           1       0.98      0.97      0.97        97

    accuracy                           0.96       112
   macro avg       0.90      0.92      0.91       112
weighted avg       0.96      0.96      0.96       112



In [35]:
# Try with custom text input and check the prediction
my_input = "Urgent! call 09066612661 from landline. Your complementary 4* Tenerife Holiday or Â£10,000 cash await collection SAE T&Cs PO Box 3 WA14 2PX 150ppm 18+ Sender: Hol Offer"

In [36]:
def processing_input(my_input):
    sms = re.sub(r'[^a-zA-Z]', ' ', my_input)
    sms = sms.lower()
    sms = sms.split()
    sms = [lemmatizer.lemmatize(word) for word in sms]
    sms = ' '.join(sms)
    sms_message = simple_preprocess(sms)
    out = avg_word2vec(sms_message)
    print(type(out))
    test_df = pd.DataFrame()
    test_df= pd.concat([test_df.astype('float64'), pd.DataFrame(out.reshape(1,-1))], ignore_index=True)
    return test_df

In [37]:
test_df = processing_input(my_input)

<class 'numpy.ndarray'>


In [38]:
test_df 

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.128238,0.255784,0.12021,0.106087,0.072697,-0.668381,0.196267,0.663593,-0.323948,-0.152009,...,0.417828,0.152295,0.014956,0.04819,0.478042,0.291574,0.072441,-0.245015,0.13953,-0.006944


In [39]:
answer = classifier.predict(test_df)

In [40]:
answer

array([0], dtype=uint8)

In [41]:
if answer[0] == 0:
    print("it is Spam")

else:
    print("It is not spam")

it is Spam


In [42]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("classifier.pkl","wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()

In [43]:
import pickle
pickle_out = open("model.pkl","wb")
pickle.dump(model, pickle_out)
pickle_out.close()